In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
import requests
host = 'https://api.census.gov/data'
year = '/2015'
dataset_acronym = '/acs/acs1'
g = '?get='
variables = 'NAME,B19083_001E'
location = '&for=state:*&key='
usr_key = "0d20bc54b2f22dccdd48690f3ef9c523a9b7da70"
query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
response = requests.get(query_url)
contents = response.text

In [3]:
contents

'[["NAME","B19083_001E","state"],\n["Mississippi","0.4759","28"],\n["Missouri","0.4634","29"],\n["Montana","0.4623","30"],\n["Nebraska","0.4473","31"],\n["Nevada","0.4548","32"],\n["New Hampshire","0.4347","33"],\n["New Jersey","0.4832","34"],\n["New Mexico","0.4795","35"],\n["New York","0.5138","36"],\n["North Carolina","0.4776","37"],\n["North Dakota","0.4664","38"],\n["Ohio","0.4637","39"],\n["Oklahoma","0.4702","40"],\n["Oregon","0.4622","41"],\n["Pennsylvania","0.4685","42"],\n["Rhode Island","0.4732","44"],\n["South Carolina","0.4698","45"],\n["South Dakota","0.4436","46"],\n["Tennessee","0.4789","47"],\n["Texas","0.4823","48"],\n["Vermont","0.4452","50"],\n["Utah","0.4252","49"],\n["Virginia","0.4681","51"],\n["Washington","0.4561","53"],\n["West Virginia","0.4581","54"],\n["Wisconsin","0.4410","55"],\n["Wyoming","0.4374","56"],\n["Puerto Rico","0.5589","72"],\n["Alabama","0.4811","01"],\n["Alaska","0.4322","02"],\n["Arizona","0.4702","04"],\n["Arkansas","0.4773","05"],\n["Calif

In [4]:
gini = pd.read_json(contents)

In [5]:
gini 

,0,1,2
0,NAME,B19083_001E,state
1,Mississippi,0.4759,28
2,Missouri,0.4634,29
3,Montana,0.4623,30
4,Nebraska,0.4473,31
5,Nevada,0.4548,32
6,New Hampshire,0.4347,33
7,New Jersey,0.4832,34
8,New Mexico,0.4795,35
9,New York,0.5138,36


In [6]:
def create_gini():
    years = ["/2011", "/2012", "/2013", "/2014", "/2015", "/2016", "/2017", "/2018", "/2019", "/2021", 
             "/2022"]
    year = "/2010"
    query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
    response = requests.get(query_url)
    contents = response.text
    gini = pd.read_json(contents)
    year_col = np.full(len(gini), year[1:])
    year_col = pd.DataFrame(year_col)
    gini = pd.concat([gini, year_col], axis = 1)
    gini.columns = ["State","gini","state", "year"]
    gini = gini[gini["State"] != "Puerto Rico"]
    gini = gini[gini["State"] != "District of Columbia"]
    for year in years:
        query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
        response = requests.get(query_url)
        contents = response.text
        temp = pd.read_json(contents)
        year_col = np.full(len(temp), year[1:])
        year_col = pd.DataFrame(year_col)
        temp = pd.concat([temp, year_col], axis = 1)
        temp.columns = ["State","gini","state", "year"]
        temp = temp.sort_values("State")
        temp = temp[temp["State"] != "District of Columbia"]
        temp = temp[temp["State"] != "Puerto Rico"]
        gini = pd.concat([gini, temp])
    return gini

In [7]:
gini = create_gini()

In [8]:
gini

,State,gini,state,year
0,NAME,B19083_001E,state,2010
1,Alabama,0.472,01,2010
2,Alaska,0.422,02,2010
3,Arizona,0.455,04,2010
4,Arkansas,0.458,05,2010
...,...,...,...,...
47,Virginia,0.4755,51,2022
48,Washington,0.4742,53,2022
49,West Virginia,0.4804,54,2022
50,Wisconsin,0.4510,55,2022


In [9]:
gini = gini[gini["State"] != "NAME"]

In [10]:
gini

,State,gini,state,year
1,Alabama,0.472,01,2010
2,Alaska,0.422,02,2010
3,Arizona,0.455,04,2010
4,Arkansas,0.458,05,2010
5,California,0.471,06,2010
...,...,...,...,...
47,Virginia,0.4755,51,2022
48,Washington,0.4742,53,2022
49,West Virginia,0.4804,54,2022
50,Wisconsin,0.4510,55,2022


In [11]:
gini.columns = ["State","gini","state", "year"]

In [12]:
gini = gini[gini["State"] != "NAME"]

In [13]:
gini

,State,gini,state,year
1,Alabama,0.472,01,2010
2,Alaska,0.422,02,2010
3,Arizona,0.455,04,2010
4,Arkansas,0.458,05,2010
5,California,0.471,06,2010
...,...,...,...,...
47,Virginia,0.4755,51,2022
48,Washington,0.4742,53,2022
49,West Virginia,0.4804,54,2022
50,Wisconsin,0.4510,55,2022


In [14]:
gini

,State,gini,state,year
1,Alabama,0.472,01,2010
2,Alaska,0.422,02,2010
3,Arizona,0.455,04,2010
4,Arkansas,0.458,05,2010
5,California,0.471,06,2010
...,...,...,...,...
47,Virginia,0.4755,51,2022
48,Washington,0.4742,53,2022
49,West Virginia,0.4804,54,2022
50,Wisconsin,0.4510,55,2022


In [15]:
gini = gini.drop(["state"], axis = 1)

In [16]:
gini

,State,gini,year
1,Alabama,0.472,2010
2,Alaska,0.422,2010
3,Arizona,0.455,2010
4,Arkansas,0.458,2010
5,California,0.471,2010
...,...,...,...
47,Virginia,0.4755,2022
48,Washington,0.4742,2022
49,West Virginia,0.4804,2022
50,Wisconsin,0.4510,2022


In [17]:
coeffs = pd.to_numeric(gini["gini"])

In [18]:
gini["gini"] = coeffs

In [19]:
years = pd.to_numeric(gini["year"])
gini["year"] = years

In [20]:
gini

,State,gini,year
1,Alabama,0.4720,2010
2,Alaska,0.4220,2010
3,Arizona,0.4550,2010
4,Arkansas,0.4580,2010
5,California,0.4710,2010
...,...,...,...
47,Virginia,0.4755,2022
48,Washington,0.4742,2022
49,West Virginia,0.4804,2022
50,Wisconsin,0.4510,2022


In [21]:
gini = gini.pivot(columns = "year", index = 'State', values = "gini")

In [22]:
gini

year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2021,2022
State,,,,,,,,,,,,
Alabama,0.472,0.4735,0.4726,0.4752,0.4758,0.4811,0.4847,0.4768,0.4859,0.4741,0.4823,0.4851
Alaska,0.422,0.4099,0.4232,0.4083,0.4175,0.4322,0.4081,0.4241,0.4322,0.4376,0.4392,0.4278
Arizona,0.455,0.4600,0.4608,0.4681,0.4676,0.4702,0.4713,0.4668,0.4613,0.4591,0.4629,0.4665
Arkansas,0.458,0.4684,0.4632,0.4685,0.4714,0.4773,0.4719,0.4725,0.4845,0.4750,0.4751,0.4799
California,0.471,0.4812,0.4822,0.4903,0.4890,0.4876,0.4899,0.4866,0.4912,0.4866,0.4924,0.4953
Colorado,0.457,0.4588,0.4579,0.4614,0.4584,0.4580,0.4586,0.4553,0.4561,0.4548,0.4604,0.4566
Connecticut,0.486,0.4859,0.4915,0.4994,0.5005,0.4916,0.4945,0.4943,0.5014,0.5024,0.4985,0.5008
Delaware,0.440,0.4404,0.4356,0.4513,0.4494,0.4523,0.4522,0.4811,0.4587,0.4509,0.4547,0.4407
Florida,0.474,0.4811,0.4828,0.4843,0.4834,0.4872,0.4852,0.4859,0.4887,0.4808,0.4869,0.4902


In [23]:
gini

year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2021,2022
State,,,,,,,,,,,,
Alabama,0.472,0.4735,0.4726,0.4752,0.4758,0.4811,0.4847,0.4768,0.4859,0.4741,0.4823,0.4851
Alaska,0.422,0.4099,0.4232,0.4083,0.4175,0.4322,0.4081,0.4241,0.4322,0.4376,0.4392,0.4278
Arizona,0.455,0.4600,0.4608,0.4681,0.4676,0.4702,0.4713,0.4668,0.4613,0.4591,0.4629,0.4665
Arkansas,0.458,0.4684,0.4632,0.4685,0.4714,0.4773,0.4719,0.4725,0.4845,0.4750,0.4751,0.4799
California,0.471,0.4812,0.4822,0.4903,0.4890,0.4876,0.4899,0.4866,0.4912,0.4866,0.4924,0.4953
Colorado,0.457,0.4588,0.4579,0.4614,0.4584,0.4580,0.4586,0.4553,0.4561,0.4548,0.4604,0.4566
Connecticut,0.486,0.4859,0.4915,0.4994,0.5005,0.4916,0.4945,0.4943,0.5014,0.5024,0.4985,0.5008
Delaware,0.440,0.4404,0.4356,0.4513,0.4494,0.4523,0.4522,0.4811,0.4587,0.4509,0.4547,0.4407
Florida,0.474,0.4811,0.4828,0.4843,0.4834,0.4872,0.4852,0.4859,0.4887,0.4808,0.4869,0.4902


In [24]:
gini = gini.reset_index()

In [25]:
gini

year,State,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2021,2022
0,Alabama,0.472,0.4735,0.4726,0.4752,0.4758,0.4811,0.4847,0.4768,0.4859,0.4741,0.4823,0.4851
1,Alaska,0.422,0.4099,0.4232,0.4083,0.4175,0.4322,0.4081,0.4241,0.4322,0.4376,0.4392,0.4278
2,Arizona,0.455,0.4600,0.4608,0.4681,0.4676,0.4702,0.4713,0.4668,0.4613,0.4591,0.4629,0.4665
3,Arkansas,0.458,0.4684,0.4632,0.4685,0.4714,0.4773,0.4719,0.4725,0.4845,0.4750,0.4751,0.4799
4,California,0.471,0.4812,0.4822,0.4903,0.4890,0.4876,0.4899,0.4866,0.4912,0.4866,0.4924,0.4953
5,Colorado,0.457,0.4588,0.4579,0.4614,0.4584,0.4580,0.4586,0.4553,0.4561,0.4548,0.4604,0.4566
6,Connecticut,0.486,0.4859,0.4915,0.4994,0.5005,0.4916,0.4945,0.4943,0.5014,0.5024,0.4985,0.5008
7,Delaware,0.440,0.4404,0.4356,0.4513,0.4494,0.4523,0.4522,0.4811,0.4587,0.4509,0.4547,0.4407
8,Florida,0.474,0.4811,0.4828,0.4843,0.4834,0.4872,0.4852,0.4859,0.4887,0.4808,0.4869,0.4902
9,Georgia,0.468,0.4770,0.4809,0.4837,0.4801,0.4801,0.4813,0.4830,0.4822,0.4795,0.4787,0.4736


In [26]:
gini.to_csv("gini_coef.csv", index = False)